Testing autosklearn

In [1]:
import numpy as np
import pandas as pd
import os
import math
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from tqdm.auto import tqdm
from autosklearn.regression import AutoSklearnRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import PredefinedSplit
import time
from sklearn.metrics import r2_score
from modAL.models import ActiveLearner


tqdm.pandas()

/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [2]:
train_df = pd.read_csv("../data/processed/PROSAIL_LUT_S2_10000_generic.csv")
validation_df = pd.read_csv("../data/processed/GBOV_RM07_in_situ_train.csv")
test_df = pd.read_csv("../data/processed/GBOV_RM07_in_situ_holdout.csv")

In [3]:
validation_df

,Unnamed: 0,Unnamed: 0.1,Site,date,plotID,retrieval_date,LAI_Miller,B1,B2,B3,...,QA20,QA60,cloud_probability,nlcdClass,observer_azimuth,observer_zenith,solar_zenith,solar_azimuth,relative_azimuth,LAI_Warren
0,607,607,LajasExperimentalStation,2016-11-23,LAJA_065,2016-11-19,1.3800,0.065434,0.070778,0.095199,...,0.0,0.0,16.0,pastureHay,102.478875,7.775917,41.609523,155.574434,53.095558,1.0600
1,2758,2758,Underc,2017-05-23,UNDE_048,2017-05-27,2.1020,0.032707,0.035183,0.054432,...,0.0,0.0,1.0,mixedForest,286.090525,8.047912,26.637376,156.032280,229.941755,2.3860
2,611,611,LajasExperimentalStation,2017-02-10,LAJA_065,2017-01-28,0.9200,0.037561,0.036632,0.066380,...,0.0,0.0,4.0,pastureHay,101.824467,7.872103,43.392796,145.979591,44.155124,0.6200
3,233,233,Jornada,2016-07-27,JORN_053,2016-07-23,0.0034,0.077465,0.097069,0.136192,...,0.0,0.0,2.0,shrubScrub,104.584100,6.829718,21.688155,122.220188,17.636088,0.0021
4,1087,1087,NorthSterling,2016-05-11,STER_014,2016-05-04,0.0371,0.094235,0.117358,0.150066,...,0.0,0.0,1.0,cultivatedCrops,211.664295,3.102768,26.866117,148.780685,297.116390,0.0570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2097,2465,2465,SteigerwaldtLandServices,2018-10-04,STEI_058,2018-10-04,3.7610,0.029406,0.032106,0.052885,...,0.0,0.0,3.0,deciduousForest,287.497226,8.950586,50.697639,168.157560,240.660334,3.2920
2098,1084,1084,NorthSterling,2015-09-02,STER_014,2015-08-28,1.1200,0.100679,0.105116,0.127461,...,0.0,0.0,23.0,cultivatedCrops,265.434501,2.434298,33.393639,151.842781,246.408281,1.5000
2099,1471,1471,OnaquiAult,2015-09-23,ONAQ_069,2015-10-06,0.0093,0.063949,0.052577,0.052016,...,0.0,0.0,30.0,shrubScrub,104.229795,9.448723,47.125692,160.904504,56.674709,0.0120
2100,1563,1563,OrdwaySwisherBiologicalStation,2018-02-01,OSBS_025,2018-01-24,0.7257,0.018663,0.035055,0.053114,...,0.0,0.0,3.0,evergreenForest,102.954880,7.695383,52.769160,153.942963,50.988083,0.5662


In [4]:
train_df = train_df.rename(columns={"lai":"LAI_Warren"})

In [5]:
import sklearn.gaussian_process

In [25]:
class KernelRidgeRegression(AutoSklearnRegressionAlgorithm):
    def __init__(self, alpha, kernel, gamma, degree, coef0, random_state=None):
        self.alpha = alpha
        self.kernel = kernel
        self.gamma = gamma
        self.degree = degree
        self.coef0 = coef0
        self.random_state = random_state
        self.estimator = None

    def fit(self, X, y):
        self.alpha = float(self.alpha)
        self.gamma = float(self.gamma)
        self.degree = int(self.degree)
        self.coef0 = float(self.coef0)

        import sklearn.kernel_ridge
        self.estimator = sklearn.kernel_ridge.KernelRidge(alpha=self.alpha,
                                                          kernel=self.kernel,
                                                          gamma=self.gamma,
                                                          degree=self.degree,
                                                          coef0=self.coef0,
                                                          )
        self.estimator.fit(X, y)
        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {'shortname': 'KRR',
                'name': 'Kernel Ridge Regression',
                'handles_regression': True,
                'handles_classification': False,
                'handles_multiclass': False,
                'handles_multilabel': False,
                'handles_multioutput': True,
                'is_deterministic': True,
                'input': (SPARSE, DENSE, UNSIGNED_DATA, SIGNED_DATA),
                'output': (PREDICTIONS,)}

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        cs = ConfigurationSpace()
        alpha = UniformFloatHyperparameter(
            name='alpha', lower=10 ** -5, upper=1, log=True, default_value=1.0)
        kernel = CategoricalHyperparameter(
            name='kernel',
            # We restrict ourselves to two possible kernels for this example
            choices=['polynomial', 'rbf'],
            default_value='polynomial'
        )
        gamma = UniformFloatHyperparameter(
            name='gamma', lower=0.00001, upper=1, default_value=0.1, log=True
        )
        degree = UniformIntegerHyperparameter(
            name='degree', lower=2, upper=5, default_value=3
        )
        coef0 = UniformFloatHyperparameter(
            name='coef0', lower=1e-2, upper=1e2, log=True, default_value=1,
        )
        cs.add_hyperparameters([alpha, kernel, gamma, degree, coef0])
        degree_condition = EqualsCondition(degree, kernel, 'polynomial')
        coef0_condition = EqualsCondition(coef0, kernel, 'polynomial')
        cs.add_conditions([degree_condition, coef0_condition])
        return cs


# Add KRR component to auto-sklearn.
autosklearn.pipeline.components.regression.add_regressor(KernelRidgeRegression)
cs = KernelRidgeRegression.get_hyperparameter_search_space()
print(cs)

Configuration space object:
  Hyperparameters:
    alpha, Type: UniformFloat, Range: [1e-05, 1.0], Default: 1.0, on log-scale
    coef0, Type: UniformFloat, Range: [0.01, 100.0], Default: 1.0, on log-scale
    degree, Type: UniformInteger, Range: [2, 5], Default: 3
    gamma, Type: UniformFloat, Range: [1e-05, 1.0], Default: 0.1, on log-scale
    kernel, Type: Categorical, Choices: {polynomial, rbf}, Default: polynomial
  Conditions:
    coef0 | kernel == 'polynomial'
    degree | kernel == 'polynomial'



/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:65: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current

In [19]:
from ConfigSpace.configuration_space import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter, \
    UniformIntegerHyperparameter, CategoricalHyperparameter
from ConfigSpace.conditions import EqualsCondition

import sklearn.metrics
import autosklearn.regression
import autosklearn.pipeline.components.regression
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm
from autosklearn.pipeline.constants import SPARSE, DENSE, \
    SIGNED_DATA, UNSIGNED_DATA, PREDICTIONS

class ActiveLearningGaussianProcessRegression(AutoSklearnRegressionAlgorithm):
    # Based on: https://github.com/automl/auto-sklearn/blob/master/autosklearn/pipeline/components/regression/gaussian_process.py
    
    def __init__(self, batch_size, n_total, alpha, thetaL, thetaU, random_state=None):
       
        # Active learner hyperparameters
        self.batch_size = batch_size
        self.n_total = n_total
        
        # GPR hyperparameters
        self.alpha = alpha
        self.thetaL = thetaL
        self.thetaU = thetaU
        
        # Other
        self.random_state = random_state
        self.estimator = None
        self.regressor = None
        self.scaler = None

    def fit(self, X, y):
        import sklearn.gaussian_process
        import numpy.random
        from modAL.models import ActiveLearner
        
        def GP_regression_std(regressor, X, batch_size):
            """
            Function query based on std predicted by regressor

            Args:
                regressor (sklearn model): Regression model type to use.
                X (np array): Training features
                batch_size (int): Number of samples to query.

            Returns:
                query_idx: indices of chosen samples
                X[query_idx]: values of chosen samples

            """

            _, std = regressor.predict(X, return_std=True)
            # Perform argmax for batch_size samples
            query_idx = np.argpartition(std, -batch_size)[-batch_size:]

            return query_idx, X[query_idx]

        self.alpha = float(self.alpha)
        self.thetaL = float(self.thetaL)
        self.thetaU = float(self.thetaU)

        n_features = X.shape[1]
        kernel = sklearn.gaussian_process.kernels.RBF(
            length_scale=[1.0]*n_features,
            length_scale_bounds=[(self.thetaL, self.thetaU)]*n_features)

        # Instantiate a Gaussian Process model
        self.regressor = sklearn.gaussian_process.GaussianProcessRegressor(
            kernel=kernel,
            n_restarts_optimizer=10,
            optimizer='fmin_l_bfgs_b',
            alpha=self.alpha,
            copy_X_train=True,
            random_state=self.random_state,
            normalize_y=True,
            )

        n_batches = int((self.n_total - self.batch_size) / self.batch_size)
        n_last_batch = self.n_total - (self.batch_size + n_batches * self.batch_size)

        initial_idx = numpy.random.choice(range(len(X)), size=self.batch_size, replace=False)
        X_training, y_training = X[initial_idx], y[initial_idx]

        self.estimator = ActiveLearner(
            estimator=self.regressor,
            query_strategy=GP_regression_std, 
            X_training=X_training, y_training=y_training
        )

        batches = [self.batch_size for i in range(n_batches)]

        if n_last_batch>0:
            batches.append(n_last_batch)

        for n_queries in batches:
            query_idx, query_instance = self.estimator.query(X, self.batch_size)
            self.estimator.teach(X[query_idx], y[query_idx])

        return self

    def predict(self, X):
        if self.estimator is None:
            raise NotImplementedError
        return self.estimator.predict(X)

    @staticmethod
    def get_properties(dataset_properties=None):
        return {'shortname': 'GPR_AL',
                'name': 'ActiveLearningGaussianProcessRegression',
                'handles_regression': True,
                'handles_classification': False,
                'handles_multiclass': False,
                'handles_multilabel': False,
                'handles_multioutput': False,
                'is_deterministic': False,
                'input': (SPARSE, DENSE, UNSIGNED_DATA, SIGNED_DATA),
                'output': (PREDICTIONS,)}

    @staticmethod
    def get_hyperparameter_search_space(dataset_properties=None):
        batch_size = UniformIntegerHyperparameter(
            name="batch_size", lower=10, upper=50, default_value=50, log=False)
        n_total = UniformIntegerHyperparameter(
            name="n_total", lower=100, upper=200, default_value=100, log=False)
        alpha = UniformFloatHyperparameter(
            name="alpha", lower=1e-14, upper=1.0, default_value=1e-8, log=True)
        thetaL = UniformFloatHyperparameter(
            name="thetaL", lower=1e-10, upper=1e-3, default_value=1e-6, log=True)
        thetaU = UniformFloatHyperparameter(
            name="thetaU", lower=1.0, upper=100000, default_value=100000.0, log=True)
        
        cs = ConfigurationSpace()
        cs.add_hyperparameters([batch_size, n_total, alpha, thetaL, thetaU])
        return cs




In [20]:
# Add GPR_AL component to auto-sklearn.
autosklearn.pipeline.components.regression.add_regressor(ActiveLearningGaussianProcessRegression)
cs = ActiveLearningGaussianProcessRegression.get_hyperparameter_search_space()
print(cs)

Configuration space object:
  Hyperparameters:
    alpha, Type: UniformFloat, Range: [1e-14, 1.0], Default: 1e-08, on log-scale
    batch_size, Type: UniformInteger, Range: [10, 50], Default: 50
    n_total, Type: UniformInteger, Range: [100, 200], Default: 100
    thetaL, Type: UniformFloat, Range: [1e-10, 0.001], Default: 1e-06, on log-scale
    thetaU, Type: UniformFloat, Range: [1.0, 100000.0], Default: 100000.0, on log-scale



/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:123: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:125: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curre

In [28]:
if __name__ == "__main__":
    autosklearn.pipeline.components.regression.add_regressor(KernelRidgeRegression)

    model = {
                "model_type":AutoSklearnRegressor,
                "hyperparams":
                {
                    "time_left_for_this_task": 1*60,
                    "per_run_time_limit":1*30,
                    "initial_configurations_via_metalearning":0,
                    "ensemble_size":1,
                    "ensemble_nbest":1,
#                     "output_folder":"/tmp/autosklearn_output",
                    "n_jobs":2,
                    "delete_tmp_folder_after_terminate":False,
#                     "delete_output_folder_after_terminate":True,
                    "include_preprocessors":["no_preprocessing"], 
                    "exclude_preprocessors":None,
                    "memory_limit":1024*6,
                    "include_estimators":["KernelRidgeRegression"]
                }
           }

    target_cols=("lai", "LAI_Warren")
    feature_cols=['B1', 'B2', 'B3', 'B4', 'B5', 'B6', \
                  'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', \
                  'solar_zenith', 'observer_zenith', 'relative_azimuth']
    standardize=False


    model["hyperparams"]["resampling_strategy"] = PredefinedSplit(test_fold=range(len(train_df), len(train_df)+len(validation_df)))

    all_cols = feature_cols.copy()
    all_cols.append(target_cols[1])

    train_data = (train_df[all_cols].append(validation_df[all_cols])).reindex()
    test_data = test_df

    # Train on entire training set
    X_train = np.array(train_data[feature_cols])
    y_train = np.array(train_data[target_cols[1]]) 

    # Shuffle training set
    # idx = np.arange(len(X_train))
    # np.random.shuffle(idx)

    # X_train = X_train[idx]
    # y_train = y_train[idx]

    X_test = np.array(test_data[feature_cols])
    y_test = np.array(test_data[target_cols[1]]) 

    # Standardize data
    if standardize:
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

    # Fit model on entire training set
    t_start = time.process_time()

    # Random Forest
    # model = {
    #     "model_type":RandomForestRegressor,
    #     "hyperparams":
    #     {
    #         "n_estimators": 100,
    #     }
    # }


    model = model["model_type"](**model["hyperparams"])

    print(model)
    model.fit(X_train, y_train)
    model.refit(X_train[:len(train_df)], y_train[:len(train_df)])
    # Get training set predictions  
    y_train_pred = model.predict(X_train)

    train_time = time.process_time()-t_start

    # Get test set predictions
    t_start = time.process_time()
    y_test_pred = model.predict(X_test)
    test_time = time.process_time()-t_start

    print(train_time)
    print(test_time)
    print(r2_score(y_test, y_test_pred))

    m = model.show_models()
    print(model.show_models())
    print(model.sprint_statistics())

AutoSklearnRegressor(delete_tmp_folder_after_terminate=False, ensemble_nbest=1,
                     ensemble_size=1,
                     include_estimators=['KernelRidgeRegression'],
                     include_preprocessors=['no_preprocessing'],
                     initial_configurations_via_metalearning=0,
                     memory_limit=6144, n_jobs=2, per_run_time_limit=30,
                     resampling_strategy=PredefinedSplit(test_fold=array([10000, 10001, ..., 12100, 12101])),
                     time_left_for_this_task=60)


/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:65: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current

48.427677367
0.0006747310000037032
-1.1378076504560326
[(1.000000, MyDummyRegressor(config=1,
                 init_params={'data_preprocessing:feat_type': {0: 'numerical',
                                                               1: 'numerical',
                                                               2: 'numerical',
                                                               3: 'numerical',
                                                               4: 'numerical',
                                                               5: 'numerical',
                                                               6: 'numerical',
                                                               7: 'numerical',
                                                               8: 'numerical',
                                                               9: 'numerical',
                                                               10: 'numerical',
                                    

In [ ]:
batch_size = UniformIntegerHyperparameter(
    name="batch_size", lower=10, upper=1500, default_value=50, log=False)
n_total = UniformIntegerHyperparameter(
    name="n_total", lower=100, upper=2000, default_value=1000, log=False)
alpha = UniformFloatHyperparameter(
    name="alpha", lower=1e-14, upper=1.0, default_value=1e-8, log=True)
thetaL = UniformFloatHyperparameter(
    name="thetaL", lower=1e-10, upper=1e-3, default_value=1e-6, log=True)
thetaU = UniformFloatHyperparameter(
    name="thetaU", lower=1.0, upper=100000, default_value=100000.0, log=True)


In [22]:
test_model = ActiveLearningGaussianProcessRegression(50, 100, 1e-8, 1e-6, 1e5)

In [23]:
test_model.fit(X_train, y_train)

TypeError: __init__() got an unexpected keyword argument 'n_jobs'

In [17]:
test_model.predict(X_test)

array([ 4.20827761e-01,  3.16907660e+00,  5.13982876e+00,  2.15780698e-01,
        3.28044838e+00,  2.73789752e+00,  3.28935383e+00,  1.97566259e+00,
        5.22953081e+00,  3.42393846e+00,  3.83016927e-04,  6.63560142e-01,
        2.67265062e+00,  9.16715284e-03,  2.88939263e+00,  2.29336090e+00,
        3.05721848e-01,  6.79985903e-01,  7.08492998e-01,  3.34438007e-03,
        3.93058329e-01,  4.60579470e+00,  3.76020487e+00,  1.76321251e+00,
        6.74150757e+00,  3.46966863e+00,  1.14194947e+00,  6.00843213e+00,
        4.09594838e+00,  2.79667327e-01,  3.34351506e+00,  2.33012349e+00,
        4.43214816e+00,  8.69891495e-02,  6.16600645e+00,  2.25519534e+00,
        5.33288852e+00,  7.66871765e+00,  8.64010465e-03,  2.40095003e+00,
        4.71498236e+00,  4.06579015e+00,  2.86628893e+00,  3.03448944e+00,
        3.84430575e+00,  4.86061788e+00,  3.46786307e+00,  2.66764154e+00,
        6.27848706e-01,  9.25075713e-01,  1.81216524e+00,  6.00377367e+00,
        4.15957710e+00, -

In [65]:
model.get_models_with_weights()[0][1].predict(X_train)

array([4., 4., 4., ..., 4., 4., 4.], dtype=float32)

In [ ]:
def train_stacked_ensemble(X_train, y_train, automl):
    X_temp = 

In [16]:
m

"[(1.000000, MyDummyRegressor(config=1, init_params={'instance': None}, random_state=1)),\n]"

In [11]:
print(model)

AutoSklearnRegressor(ensemble_nbest=1, ensemble_size=1,
                     include_preprocessors=['no_preprocessing'],
                     initial_configurations_via_metalearning=0, n_jobs=2,
                     output_folder='/tmp/autosklearn_output',
                     per_run_time_limit=60,
                     resampling_strategy=<class 'sklearn.model_selection._split.PredefinedSplit'>,
                     resampling_strategy_arguments={'test_fold': range(10000, 12102)},
                     time_left_for_this_task=300,
                     tmp_folder='/tmp/autosklearn_tmp')


In [17]:
24*60*60

86400

In [6]:
from sklearn.base import BaseEstimator 
from sklearn.base import RegressorMixin

In [11]:
class AutoMLStackingRegressor(BaseEstimator, RegressorMixin):

    def __init__(self, automl, final_estimator, **model_hyper_parameters):
        """
        Works similar to sklearn's StackingRegressor, 
        except that the input estimators are not refitted during a call to fit().

        param automl: Fitted auto-sklearn object, from which estimator pipelines can be extracted
        param final_estimator: Untrained sklearn estimator
        """
        super().__init__()
        self.automl = automl
        self.final_estimator = final_estimator
        self.is_fit=False
    
    def _get_estimators(self):
        """
        Extracts estimators from autosklearn object
        """
        
        
        self.estimators = [m[1] for m in self.automl.get_models_with_weights()]
        return
    
    def _get_X_final_model(self, X):
        """
        Adds predictions from autosklearn estimators as features in X
        """
    
        y_pred = np.array([estimator.predict(X) for estimator in self.estimators]).reshape(-1,1)
        X_final_model = np.c_[X, y_pred]  
        return X_final_model


    def fit(self, X, y):
        self._get_estimators()
        
        X_final_model = self._get_X_final_model(X)

        self.final_estimator.fit(X_final_model, y)
        self._fit=True
        return self

    def predict(self, X):
        assert(self._fit==True), "Unable to predict. Call the fit() function first!"

        X_final_model = self._get_X_final_model(X)

        y_pred = self.final_estimator.predict(X_final_model)

        return y_pred

In [12]:
final_model = AutoMLStackingRegressor(automl=model, final_estimator=RandomForestRegressor())

In [13]:
final_model.fit(X_train[len(train_df):], y_train[len(train_df):])

AutoMLStackingRegressor(automl=AutoSklearnRegressor(ensemble_nbest=1,
                                                    ensemble_size=1,
                                                    include_preprocessors=['no_preprocessing'],
                                                    initial_configurations_via_metalearning=0,
                                                    n_jobs=2,
                                                    output_folder='/tmp/autosklearn_output',
                                                    per_run_time_limit=60,
                                                    resampling_strategy=<class 'sklearn.model_selection._split.PredefinedSplit'>,
                                                    resampling_strategy_arguments={'test_fold': range(10000, 12102)},
                                                    time_left_for_this_task=300,
                                                    tmp_folder='/tmp/autosklearn_tmp'),
                       

In [14]:
final_model.score(X_test, y_test)

0.8639054640853376

In [24]:
train_df.describe()

,Unnamed: 0,n,cab,car,cbrown,cw,cm,LAI_Warren,lidfa,hspot,...,B4,B5,B6,B7,B8,B8A,B9,B10,B11,B12
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4999.50000,1.750000,50.000000,8.500000,1.000000,0.100000,0.100000,4.000000,45.000000,0.075000,...,0.036183,0.067180,0.129195,0.154254,0.167312,0.174060,0.171832,0.082803,0.066721,0.042489
std,2886.89568,0.433034,17.321374,4.330344,0.577379,0.057738,0.057738,2.309517,23.095165,0.014434,...,0.037206,0.040281,0.064657,0.082582,0.091378,0.096391,0.092803,0.068753,0.067625,0.062521
min,0.00000,1.000075,20.003000,1.000750,0.000100,0.000010,0.000010,0.000400,5.004000,0.050002,...,0.001574,0.006678,0.014935,0.017731,0.019403,0.020250,0.020066,0.005064,0.003006,0.000884
25%,2499.75000,1.375037,35.001500,4.750375,0.500050,0.050005,0.050005,2.000200,25.002000,0.062501,...,0.017102,0.038939,0.081280,0.094391,0.100939,0.104070,0.102842,0.035046,0.023089,0.009617
50%,4999.50000,1.750000,50.000000,8.500000,1.000000,0.100000,0.100000,4.000000,45.000000,0.075000,...,0.022311,0.056801,0.118740,0.139087,0.149228,0.154161,0.153295,0.058385,0.039608,0.013295
75%,7499.25000,2.124963,64.998500,12.249625,1.499950,0.149995,0.149995,5.999800,64.998000,0.087499,...,0.034553,0.083503,0.167918,0.198793,0.215923,0.224203,0.224990,0.105039,0.080556,0.041685
max,9999.00000,2.499925,79.997000,15.999250,1.999900,0.199990,0.199990,7.999600,84.996000,0.099998,...,0.200144,0.220337,0.598948,0.793209,0.779937,0.765597,0.778564,0.725032,0.701275,0.530866


In [25]:
test_df.describe()

,Unnamed: 0,Unnamed: 0.1,LAI_Miller,B1,B2,B3,B4,B5,B6,B7,...,QA10,QA20,QA60,cloud_probability,observer_azimuth,observer_zenith,solar_zenith,solar_azimuth,relative_azimuth,LAI_Warren
count,902.000000,902.000000,902.000000,902.000000,902.000000,902.000000,902.000000,902.000000,902.000000,902.000000,...,902.0,902.0,902.000000,902.000000,902.000000,902.000000,902.000000,902.000000,902.000000,902.000000
mean,1468.145233,1468.145233,2.162702,0.057510,0.065264,0.087519,0.096007,0.136755,0.242238,0.277372,...,0.0,0.0,145.312639,8.004435,187.459566,5.978706,31.082048,143.645532,170.110579,1.859110
std,874.844624,874.844624,2.386208,0.069278,0.071170,0.069091,0.088034,0.080104,0.078004,0.092220,...,0.0,0.0,501.186567,14.095441,77.317421,2.129161,10.165473,16.803007,113.879092,2.008171
min,5.000000,5.000000,0.000031,-0.106119,-0.052211,-0.016778,-0.008774,0.010087,0.019968,0.024013,...,0.0,0.0,0.000000,0.000000,101.050870,2.910594,15.949376,72.774354,0.946588,0.000051
25%,691.000000,691.000000,0.056700,0.024952,0.028020,0.044474,0.030350,0.079542,0.199428,0.222389,...,0.0,0.0,0.000000,1.000000,104.221948,4.258278,22.477422,136.262534,45.150719,0.060250
50%,1447.000000,1447.000000,1.220000,0.047682,0.051702,0.073660,0.069721,0.117938,0.237712,0.264967,...,0.0,0.0,0.000000,2.000000,210.274719,5.618022,28.704009,147.225822,225.739016,1.067000
75%,2231.250000,2231.250000,4.033750,0.076814,0.091615,0.121039,0.156133,0.187359,0.282174,0.320273,...,0.0,0.0,0.000000,6.000000,256.246612,7.732823,37.373013,154.686462,256.762344,3.591500
max,3003.000000,3003.000000,10.980000,1.119873,1.096225,1.013699,1.011749,1.043744,1.023045,0.994278,...,0.0,0.0,2048.000000,68.000000,288.279895,10.032018,58.763682,170.783442,359.986992,7.970000


In [34]:
target_cols=("lai", "LAI_Warren")
feature_cols=['B1', 'B2', 'B3', 'B4', 'B5', 'B6', \
              'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', \
              'solar_zenith', 'observer_zenith', 'relative_azimuth']

all_cols = feature_cols.copy()
all_cols.append(target_cols[1])

train_data = (train_df[all_cols].append(validation_df[all_cols])).reindex()
test_data = test_df

# Train on entire training set
X_train = np.array(train_data[feature_cols])
y_train = np.array(train_data[target_cols[1]]) 
X_test = np.array(test_data[feature_cols])
y_test = np.array(test_data[target_cols[1]]) 

In [27]:
print(feature_cols)

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'solar_zenith', 'observer_zenith', 'relative_azimuth']


In [37]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
r2_score(y_true=y_test, y_pred=rf_pred)

0.830743749385262

In [29]:
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.030222,0.034490,0.048469,0.036183,0.067180,0.129195,0.154254,0.167312,0.174060,0.171832,0.066721,0.042489,37.500000,5.000000,180.000000
std,0.024853,0.025900,0.030759,0.037206,0.040281,0.064657,0.082582,0.091378,0.096391,0.092803,0.067625,0.062521,12.991031,2.886896,103.928244
min,0.001430,0.001671,0.003785,0.001574,0.006678,0.014935,0.017731,0.019403,0.020250,0.020066,0.003006,0.000884,15.002250,0.000500,0.018000
25%,0.017477,0.020150,0.027911,0.017102,0.038939,0.081280,0.094391,0.100939,0.104070,0.102842,0.023089,0.009617,26.251125,2.500250,90.009000
50%,0.022934,0.026697,0.039714,0.022311,0.056801,0.118740,0.139087,0.149228,0.154161,0.153295,0.039608,0.013295,37.500000,5.000000,180.000000
75%,0.029193,0.036663,0.059188,0.034553,0.083503,0.167918,0.198793,0.215923,0.224203,0.224990,0.080556,0.041685,48.748875,7.499750,269.991000
max,0.139955,0.146249,0.174141,0.200144,0.220337,0.598948,0.793209,0.779937,0.765597,0.778564,0.701275,0.530866,59.997750,9.999500,359.982000
